In [1]:
from astropy.io import fits
from astropy.table import unique, vstack
from astropy.time import Time
from astroquery.mast import Mast,Observations
from astropy.table import Table
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import json

In [2]:
def set_params(parameters):
    return [{"paramName" : p, "values" : v} for p, v in parameters.items()]

def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }


In [5]:
def get_proposal_products(proposal_id:str):
    
    service = 'Mast.Jwst.Filtered.Miri'

    keywords = {
            'exp_type':['MIR_IMAGE'],
            'instrume':['MIRI'],
            'date_obs_mjd': [set_mjd_range('2022-06-01','2022-12-01')],
           }

    params = {'columns': '*',
          'filters': set_params(keywords)
         }

    
    t = Mast.service_request(service, params)
    fn = list(set(t['filename']))
    ids = list(set(['_'.join(x.split('_')[:-1]) for x in fn]))
    
    matched_obs = Observations.query_criteria(instrument_name='Miri',
                                          obs_id=ids,
                                         )
    batch_size = 10
    batches = [matched_obs[i:i+batch_size] for i in range(0, len(matched_obs), batch_size)]
    t = [Observations.get_product_list(obs) for obs in batches]
    products = unique(vstack(t), keys='productFilename')
    filtered_products = Observations.filter_products(products,
                                                 #dataproduct_type='image',
                                                 #productType = ['SCIENCE'],
                                                 #proposal_id = [f'{proposal_id}'],)
                                                 productSubGroupDescription = ['RATEINTS'])
                                                                               #'CALINTS',
                                                                               #'ASN'])
    
    return products,filtered_products

In [6]:
products, filtered_products = get_proposal_products(proposal_id='1386')

In [7]:
filtered_products

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str9,str4,str5,str42,str64,str1,str77,str9,str28,str13,str1,str7,str5,str4,str59,int64,str9,str16,int64
87529899,JWST,image,jw01022017001_03101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw01022017001_03101_00001_mirimage_rateints.fits,SCIENCE,--,RATEINTS,--,CALJWST,1.5.3,1022,jw01022017001_03101_00001_mirimage_rateints.fits,42336000,87529921,PUBLIC,2
87529912,JWST,image,jw01022018001_02101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw01022018001_02101_00001_mirimage_rateints.fits,SCIENCE,--,RATEINTS,--,CALJWST,1.5.3,1022,jw01022018001_02101_00001_mirimage_rateints.fits,42336000,87529944,PUBLIC,2
87529927,JWST,image,jw01022019001_02101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw01022019001_02101_00001_mirimage_rateints.fits,SCIENCE,--,RATEINTS,--,CALJWST,1.5.3,1022,jw01022019001_02101_00001_mirimage_rateints.fits,42336000,87529958,PUBLIC,2
87584571,JWST,image,jw01022020001_03101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw01022020001_03101_00001_mirimage_rateints.fits,SCIENCE,--,RATEINTS,--,CALJWST,1.5.3,1022,jw01022020001_03101_00001_mirimage_rateints.fits,42336000,87584603,PUBLIC,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88594572,JWST,image,jw02741021001_02101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw02741021001_02101_00001_mirimage_rateints.fits,SCIENCE,Minimum Recommended Products,RATEINTS,--,CALJWST,1.5.3,2741,jw02741021001_02101_00001_mirimage_rateints.fits,21193920,88594572,PUBLIC,2
89364685,JWST,image,jw02741022001_02101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw02741022001_02101_00001_mirimage_rateints.fits,SCIENCE,Minimum Recommended Products,RATEINTS,--,CALJWST,1.5.3,2741,jw02741022001_02101_00001_mirimage_rateints.fits,21193920,89364685,PUBLIC,2
89375409,JWST,image,jw02741023001_02101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw02741023001_02101_00001_mirimage_rateints.fits,SCIENCE,Minimum Recommended Products,RATEINTS,--,CALJWST,1.6.2,2741,jw02741023001_02101_00001_mirimage_rateints.fits,21196800,89375409,PUBLIC,2
89376489,JWST,image,jw02741024001_02101_00001_mirimage,exposure (L2a): 3D countrate per integration,S,mast:JWST/product/jw02741024001_02101_00001_mirimage_rateints.fits,SCIENCE,Minimum Recommended Products,RATEINTS,--,CALJWST,1.6.2,2741,jw02741024001_02101_00001_mirimage_rateints.fits,21196800,89376489,PUBLIC,2


In [9]:
def determine_and_count(column:str,table):    
    prop_list = []
    counter = {}
    for i in range(len(table['proposal_id'])):

        if table[f'{column}'][i] not in prop_list:

            prop_list.append(table[f'{column}'][i])
            try:
                counter[table[f'{column}'][i]] = 1
            except:
                print(table[f'{column}'][i])
                continue
        else:
            try:
                counter[table[f'{column}'][i]] += 1
            except:
                continue
            
    print(prop_list)
    print(counter)

In [10]:
determine_and_count(column='proposal_id',table=products)

['1045', '1532', '1536', '1538']
{'1045': 1428, '1532': 1474, '1536': 1274, '1538': 1007}


In [23]:
determine_and_count(column='productSubGroupDescription',table=filtered_products)

['ASN', 'RATEINTS']
{'ASN': 266, 'RATEINTS': 319}
